### Semana 1
A2C puro para debug e validação do pipeline.

### Semana 2
PPO sem GAE para estabilizar updates.

> **Essa é a Semana 2**

PPO significa *Proximal Policy Optimization*, um método de treinamento usado em aprendizado por reforço. A ideia é treinar um agente, tipo um robô virtual ou um modelo de linguagem, para tomar decisões que rendem melhores resultados ao longo do tempo. Ele faz isso ajustando a política do agente, mas sem deixar as mudanças malucas demais, porque modelos adoram pirar quando você deixa solto.

PPO controla mudanças descontroladas usando duas estratégias:

#### 1. Clipping da razão das políticas
- Ele compara a nova política com a antiga, transformando isso em uma razão: `prob_nova / prob_antiga`.
- Se essa razão tenta crescer demais, o algoritmo corta.
- Isso impede que o modelo dê saltos gigantes que quase sempre acabam em desastre.

#### 2. Função de perda que pune exageros
- Além do clipping, há uma função de perda que desincentiva atualizações agressivas.
- Mesmo que o modelo tente inovar de um jeito duvidoso, ele recebe uma punição bem didática.


O que realmente acontece com batches pequenos

Quando você atualiza a rede com todos os 256 dados de uma vez, você calcula um gradiente muito estável, super suave, super “matematicamente bonitinho”.

Mas:

fica pesado computacionalmente

a rede dá poucos updates por rollout (porque cada update é custoso)

Quando você usa minibatches, tipo 64:

você divide em 4 atualizações menores

cada update é mais leve

mas cada um tem um pouco mais de ruído (porque só vê parte dos dados)

Esse ruído, por incrível que pareça, não é ruim.
Ele ajuda a escapar de mínimos ruins e evita que a rede fique certinha demais em cima do rollout atual.

É o mesmo motivo de SGD ser usado no mundo inteiro em vez de “usar todos os dados de uma vez”.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
import numpy as np

from rl_env import PortfolioEnv, PolicyMLP, ValueMLP
from app import (
    ret, r_media, v_media, dd,
    regime_ids, cluster_ids, NUM_ATIVOS,
    aplicar_acao_portfolio
)


#? Hiperparâmetros

AÇÕES_POR_ATIVOS = 2
num_actions = NUM_ATIVOS * AÇÕES_POR_ATIVOS

state_dim = NUM_ATIVOS + 3 + NUM_ATIVOS # pesos + (r_media, v_media, dd) + clusters

#A política é só a função que diz:dado um estado → qual probabilidade dou para cada ação

# quão neurótico o PPO vai ser durante o treino.
gamma = 0.99 # Diz o quanto o agente valoriza recompensas futuras.
clip_eps = 0.2 # É o “clipe” do PPO, a margem de tolerância de mudança da política.Com 0.2, você deixa a política mudar no máximo ~20% por atualização (em termos de probabilidade relativa)
ppo_epochs = 4           # quantas vezes reaproveitar o mesmo rollout
rollout_len = 256        # passos por iteração de PPO
batch_size = 64 # Você pega esses 256 dados e treina a rede em 4 blocos de 64 cada vez
lr = 1e-4 # Taxa de aprendizado do Adam.Quando a rede neural vê um erro (loss), ela tenta ajustar os pesos pra diminuir esse erro.
entropy_coef = 0.01 # Entropia alta significa que a política está mais “aleatória”, explorando mais.Se for 0, o agente tende a ficar determinístico muito rápido e pode travar em política ruim.
value_coef = 0.5 # Peso da loss da função de valor na loss total. loss = policy_loss + value_coef * value_loss - entropy_coef * entropy

#? Modelos
policy = PolicyMLP(state_dim, num_actions)
value = ValueMLP(state_dim)

def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.zeros_(m.bias)

policy.apply(init_weights)
value.apply(init_weights)

#value.parameters() = “tudo que deve ser treinado dentro da rede value” e outro parametro é otimizado adam 
optimizerP = optim.Adam(policy.parameters(), lr=lr)
optimizerV = optim.Adam(value.parameters(), lr=lr)

#? Ambiente
env = PortfolioEnv(ret, r_media, v_media, dd, regime_ids, cluster_ids)

#? Função auxiliares 
def compute_returns_and_advantages(rewards, dones, values, gamma=0.99):
    """
    PPO sem GAE:
    - returns = soma de recompensas descontadas (MC)
    - advantages = returns - values
    """
    rewards = np.array(rewards, dtype=np.float32)
    dones = np.array(dones, dtype=np.float32)
    values = np.array(values, dtype=np.float32)

    T = len(rewards)
    returns = np.zeros(T, dtype=np.float32)
    running_return = 0.0

    for t in reversed(range(T)):
        if dones[t] == 1.0:
            running_return = 0.0
        running_return = rewards[t] + gamma * running_return
        returns[t] = running_return

    advantages = returns - values
    # normaliza advantage pra não ficar tudo minúsculo
    adv_mean = advantages.mean()
    adv_std = advantages.std() + 1e-8
    advantages = (advantages - adv_mean) / adv_std

    return returns, advantages



Arquivo encontrado! Carregando dados locais...
            BOVA11.SA  SMAL11.SA  IVVB11.SA  PETR4.SA   VALE3.SA  ITUB4.SA  \
Date                                                                         
2015-01-01        NaN        NaN        NaN       NaN        NaN       NaN   
2015-01-02  47.259998  52.020000  55.799999  2.572509  10.505502  9.372999   
2015-01-03        NaN        NaN        NaN       NaN        NaN       NaN   
2015-01-04        NaN        NaN        NaN       NaN        NaN       NaN   
2015-01-05  46.320000  50.549999  55.750000  2.352637  10.347521  9.420101   

                   SPY        QQQ         IWM        EEM         GLD  \
Date                                                                   
2015-01-01         NaN        NaN         NaN        NaN         NaN   
2015-01-02  171.093704  94.906532  103.315140  30.789457  114.080002   
2015-01-03         NaN        NaN         NaN        NaN         NaN   
2015-01-04         NaN        NaN         NaN 

c:\Users\olazu\OneDrive\Documentos\Python\portfolioDeAtivos\portfolioDeAtivos\teste.py:154: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  retornos = precos.pct_change().fillna(0)


state shape: torch.Size([1, 37])
action_probs shape: torch.Size([1, 34])
sum probs: 1.0000001192092896
Ep 0 | reward -0.00004 | regime neutro
Ep 200 | reward -0.01729 | regime neutro
Ep 400 | reward -0.10698 | regime alta_vol
Ep 600 | reward 0.00055 | regime bull
Ep 800 | reward -0.00024 | regime bull
Ep 1000 | reward 0.00074 | regime bull
Ep 1200 | reward -0.05478 | regime alta_vol
Ep 1400 | reward -0.16539 | regime alta_vol
Ep 1600 | reward -0.01537 | regime alta_vol
Ep 1800 | reward -0.04339 | regime neutro
Ep 2000 | reward -0.00163 | regime alta_vol
Ep 2200 | reward 0.01858 | regime bull
Ep 2400 | reward -0.01316 | regime alta_vol
Ep 2600 | reward -0.09552 | regime alta_vol
Ep 2800 | reward -0.44242 | regime bear
Ep 3000 | reward -0.39315 | regime bear
Ep 3200 | reward -0.30454 | regime bear
Ep 3400 | reward -0.01185 | regime alta_vol
Ep 3600 | reward 0.01152 | regime bull
Ep 3800 | reward -0.00657 | regime alta_vol


In [2]:
# ? Loop de treino
num_iterations = 50   # iterações de PPO (cada uma com 1 rollout + vários updates)
episodio_global = 0   # contador de episódios (igual A2C)

# reset só uma vez, fora do loop
state = env.reset()
state = torch.FloatTensor(state).unsqueeze(0)

ep_reward = 0.0  # recompensa acumulada do episódio atual

for it in range(num_iterations):
    states_buf = []
    actions_buf = []
    logprobs_buf = []
    rewards_buf = []
    dones_buf = []
    values_buf = []

    # 1) COLETA DE TRAJETÓRIA (ROLLOUT)
    for step in range(rollout_len):
        # policy -> logits
        logits = policy(state)

        if torch.isnan(logits).any() or not torch.isfinite(logits).all():
            print("DEU RUIM: logits inválidos (NaN/Inf)")
            print("state:", state)
            print("logits:", logits)
            raise RuntimeError("Logits NaN/Inf em PPO")

        dist = Categorical(logits=logits)
        action = dist.sample()
        log_prob = dist.log_prob(action)

        # value atual
        V = value(state)

        # aplica ação -> novos pesos
        novos_pesos = aplicar_acao_portfolio(env.pesos, action.item())

        # passo no ambiente
        next_state, reward, done = env.step(novos_pesos)

        # acumula recompensa do episódio (para log tipo A2C)
        ep_reward += reward

        # guarda no buffer (em numpy, sem batch)
        states_buf.append(state.squeeze(0).detach().numpy())
        actions_buf.append(action.item())
        logprobs_buf.append(log_prob.item())
        rewards_buf.append(reward)
        dones_buf.append(float(done))
        values_buf.append(V.item())

        # prepara próximo estado
        state = torch.FloatTensor(next_state).unsqueeze(0)

        if done:
            # log igual ao A2C
            print(f"Episódio {episodio_global} -> reward total = {ep_reward:.5f}")
            episodio_global += 1
            ep_reward = 0.0  # zera para o próximo episódio

            # começa novo episódio
            state = env.reset()
            state = torch.FloatTensor(state).unsqueeze(0)

    # 2) CONVERTE BUFFER EM TENSORES
    states = torch.FloatTensor(np.array(states_buf))            # [T, state_dim]
    actions = torch.LongTensor(np.array(actions_buf))           # [T]
    old_logprobs = torch.FloatTensor(np.array(logprobs_buf))    # [T]
    rewards = np.array(rewards_buf, dtype=np.float32)           # [T]
    dones = np.array(dones_buf, dtype=np.float32)               # [T]
    values_arr = np.array(values_buf, dtype=np.float32)         # [T]

    # 3) RETURNS + ADVANTAGES (sem GAE)
    returns_arr, advantages_arr = compute_returns_and_advantages(
        rewards, dones, values_arr, gamma=gamma
    )

    returns = torch.FloatTensor(returns_arr)
    advantages = torch.FloatTensor(advantages_arr)

    # 4) PPO UPDATES (várias epochs sobre a mesma trajetória)
    dataset_size = states.size(0)
    idxs = np.arange(dataset_size)

    for epoch in range(ppo_epochs):
        np.random.shuffle(idxs)

        for start in range(0, dataset_size, batch_size):
            end = start + batch_size
            batch_idx = idxs[start:end]

            batch_states = states[batch_idx]
            batch_actions = actions[batch_idx]
            batch_old_logprobs = old_logprobs[batch_idx]
            batch_returns = returns[batch_idx]
            batch_advantages = advantages[batch_idx]

            # policy nova (π_new)
            logits_new = policy(batch_states)
            dist_new = Categorical(logits=logits_new)
            new_logprobs = dist_new.log_prob(batch_actions)
            entropy = dist_new.entropy().mean()

            # ratio = π_new / π_old
            ratios = torch.exp(new_logprobs - batch_old_logprobs)

            surr1 = ratios * batch_advantages
            surr2 = torch.clamp(ratios, 1.0 - clip_eps, 1.0 + clip_eps) * batch_advantages
            policy_loss = -torch.min(surr1, surr2).mean()

            # value loss (MSE entre V e returns)
            V_pred = value(batch_states).squeeze(-1)
            value_loss = nn.functional.mse_loss(V_pred, batch_returns)

            # perda total
            loss = policy_loss + value_coef * value_loss - entropy_coef * entropy

            optimizerP.zero_grad()
            optimizerV.zero_grad()
            loss.backward()
            optimizerP.step()
            optimizerV.step()

    # 5) LOG BÁSICO (nível de rollout/iteração)
    # Se você quiser ficar mais parecido com o A2C, pode logar a soma do rollout:
    total_rollout_reward = rewards.sum()
    avg_reward = rewards.mean()
    avg_return = returns.mean().item()
    avg_adv = advantages.mean().item()

    print(f"[PPO] Iter {it} | "
          f"rollout_reward={total_rollout_reward:.6f} | "
          f"avg_reward={avg_reward:.6f} | "
          f"avg_return={avg_return:.6f} | "
          f"avg_adv={avg_adv:.4f}")




[PPO] Iter 0 | rollout_reward=0.046033 | avg_reward=0.000180 | avg_return=0.006202 | avg_adv=0.0000
[PPO] Iter 1 | rollout_reward=0.058886 | avg_reward=0.000230 | avg_return=0.026984 | avg_adv=0.0000
[PPO] Iter 2 | rollout_reward=0.289213 | avg_reward=0.001130 | avg_return=0.068740 | avg_adv=0.0000
[PPO] Iter 3 | rollout_reward=0.231293 | avg_reward=0.000903 | avg_return=0.066080 | avg_adv=0.0000
[PPO] Iter 4 | rollout_reward=0.438638 | avg_reward=0.001713 | avg_return=0.057981 | avg_adv=-0.0000
[PPO] Iter 5 | rollout_reward=0.083741 | avg_reward=0.000327 | avg_return=0.028280 | avg_adv=0.0000
[PPO] Iter 6 | rollout_reward=0.130051 | avg_reward=0.000508 | avg_return=0.008649 | avg_adv=-0.0000
[PPO] Iter 7 | rollout_reward=0.401650 | avg_reward=0.001569 | avg_return=0.139654 | avg_adv=0.0000
[PPO] Iter 8 | rollout_reward=1.248102 | avg_reward=0.004875 | avg_return=0.391111 | avg_adv=-0.0000
[PPO] Iter 9 | rollout_reward=0.308719 | avg_reward=0.001206 | avg_return=0.065174 | avg_adv=0.00

PPO + GEN Semana 3 Modelo para Produção

In [1]:
# PPO + GEN Semana 3 Modelo para Produção

import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
import numpy as np
import random
import importlib
import teste  # importa o módulo
importlib.reload(teste)
from rl_env import PortfolioEnv, PolicyMLP, ValueMLP
from teste import (
    ret_train, r_media_train, v_media_train, dd_train, regime_ids_train,
    ret_test, r_media_test, v_media_test, dd_test, regime_ids_test,
    cluster_ids, NUM_ATIVOS,
    aplicar_acao_portfolio
)

SEED = 5
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# Para deixar o PyTorch mais determinístico:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# =========================
# Hiperparâmetros
# =========================

AÇÕES_POR_ATIVOS = 2
num_actions = NUM_ATIVOS * AÇÕES_POR_ATIVOS

state_dim = NUM_ATIVOS + 3 + NUM_ATIVOS  # pesos + (r_media, v_media, dd) + clusters

gamma = 0.99       # desconto das recompensas futuras
clip_eps = 0.2     # clipping do PPO
ppo_epochs = 4     # quantas vezes reaproveitar o mesmo rollout
rollout_len = 256  # passos por iteração de PPO
batch_size = 64    # tamanho do minibatch
lr = 1e-4
entropy_coef = 0.01
value_coef = 0.5

# =========================
# Modelos
# =========================
policy = PolicyMLP(state_dim, num_actions)
value = ValueMLP(state_dim)

def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.zeros_(m.bias)

policy.apply(init_weights)
value.apply(init_weights)

optimizerP = optim.Adam(policy.parameters(), lr=lr)
optimizerV = optim.Adam(value.parameters(), lr=lr)

# =========================
# Ambiente de TREINO (só dados de treino)
# =========================
env_train = PortfolioEnv(
    ret_train,
    r_media_train,
    v_media_train,
    dd_train,
    regime_ids_train,
    cluster_ids
)

# =========================
# Função auxiliar: GAE
# =========================
def compute_gae(rewards, values, dones, last_value, gamma=0.99, lam=0.95):
    """
    GAE(λ) - Generalized Advantage Estimation
    """
    rewards = np.array(rewards, dtype=np.float32)
    values = np.array(values, dtype=np.float32)
    dones = np.array(dones, dtype=np.float32)

    values_ext = np.append(values, last_value)

    T = len(rewards)
    advantages = np.zeros(T, dtype=np.float32)
    gae = 0.0

    for t in reversed(range(T)):
        delta = rewards[t] + gamma * values_ext[t + 1] * (1.0 - dones[t]) - values_ext[t]
        gae = delta + gamma * lam * (1.0 - dones[t]) * gae
        advantages[t] = gae

    returns = advantages + values

    adv_mean = advantages.mean()
    adv_std = advantages.std() + 1e-8
    advantages = (advantages - adv_mean) / adv_std

    return returns, advantages


# =========================
# Loop de TREINO (PPO + GAE) – só no TREINO
# =========================
num_iterations = 50   # iterações de PPO (cada uma com 1 rollout + vários updates)
episodio_global = 0

# reset só uma vez, fora do loop, no ambiente de treino
state = env_train.reset()
state = torch.FloatTensor(state).unsqueeze(0)

ep_reward = 0.0

for it in range(num_iterations):
    states_buf = []
    actions_buf = []
    logprobs_buf = []
    rewards_buf = []
    dones_buf = []
    values_buf = []

    # 1) COLETA DE TRAJETÓRIA (ROLLOUT) no ENV_TRAIN
    for step in range(rollout_len):
        logits = policy(state)

        if torch.isnan(logits).any() or not torch.isfinite(logits).all():
            print("DEU RUIM: logits inválidos (NaN/Inf)")
            print("state:", state)
            print("logits:", logits)
            raise RuntimeError("Logits NaN/Inf em PPO")

        dist = Categorical(logits=logits)
        action = dist.sample()
        log_prob = dist.log_prob(action)

        V = value(state)

        # aplica ação -> novos pesos usando o ambiente de TREINO
        novos_pesos = aplicar_acao_portfolio(env_train.pesos, action.item())

        next_state, reward, done = env_train.step(novos_pesos)

        ep_reward += reward

        states_buf.append(state.squeeze(0).detach().numpy())
        actions_buf.append(action.item())
        logprobs_buf.append(log_prob.item())
        rewards_buf.append(reward)
        dones_buf.append(float(done))
        values_buf.append(V.item())

        state = torch.FloatTensor(next_state).unsqueeze(0)

        if done:
            print(f"Episódio {episodio_global} -> reward total = {ep_reward:.5f}")
            episodio_global += 1
            ep_reward = 0.0

            state = env_train.reset()
            state = torch.FloatTensor(state).unsqueeze(0)

    # 2) CONVERTE BUFFER EM TENSORES / ARRAYS
    states = torch.FloatTensor(np.array(states_buf))
    actions = torch.LongTensor(np.array(actions_buf))
    old_logprobs = torch.FloatTensor(np.array(logprobs_buf))
    rewards = np.array(rewards_buf, dtype=np.float32)
    dones = np.array(dones_buf, dtype=np.float32)
    values_arr = np.array(values_buf, dtype=np.float32)

    # pega V(s_{T+1}) com o último state do rollout
    with torch.no_grad():
        last_value = value(state).item()

    # 3) RETURNS + ADVANTAGES com GAE
    returns_arr, advantages_arr = compute_gae(
        rewards, values_arr, dones, last_value, gamma=gamma, lam=0.95
    )

    returns = torch.FloatTensor(returns_arr)
    advantages = torch.FloatTensor(advantages_arr)

    # 4) PPO UPDATES
    dataset_size = states.size(0)
    idxs = np.arange(dataset_size)

    for epoch in range(ppo_epochs):
        np.random.shuffle(idxs)

        for start in range(0, dataset_size, batch_size):
            end = start + batch_size
            batch_idx = idxs[start:end]

            batch_states = states[batch_idx]
            batch_actions = actions[batch_idx]
            batch_old_logprobs = old_logprobs[batch_idx]
            batch_returns = returns[batch_idx]
            batch_advantages = advantages[batch_idx]

            logits_new = policy(batch_states)
            dist_new = Categorical(logits=logits_new)
            new_logprobs = dist_new.log_prob(batch_actions)
            entropy = dist_new.entropy().mean()

            ratios = torch.exp(new_logprobs - batch_old_logprobs)

            surr1 = ratios * batch_advantages
            surr2 = torch.clamp(ratios, 1.0 - clip_eps, 1.0 + clip_eps) * batch_advantages
            policy_loss = -torch.min(surr1, surr2).mean()

            V_pred = value(batch_states).squeeze(-1)
            value_loss = nn.functional.mse_loss(V_pred, batch_returns)

            loss = policy_loss + value_coef * value_loss - entropy_coef * entropy

            optimizerP.zero_grad()
            optimizerV.zero_grad()
            loss.backward()
            optimizerP.step()
            optimizerV.step()

    total_rollout_reward = rewards.sum()
    avg_reward = rewards.mean()
    avg_return = returns.mean().item()
    avg_adv = advantages.mean().item()

    print(f"[PPO] Iter {it} | "
          f"rollout_reward={total_rollout_reward:.6f} | "
          f"avg_reward={avg_reward:.6f} | "
          f"avg_return={avg_return:.6f} | "
          f"avg_adv={avg_adv:.4f}")


# =========================
# AVALIAÇÃO NO CONJUNTO DE TESTE (sem treinar!)
# =========================

env_test = PortfolioEnv(
    ret_test,
    r_media_test,
    v_media_test,
    dd_test,
    regime_ids_test,
    cluster_ids
)

state = env_test.reset()
state = torch.FloatTensor(state).unsqueeze(0)

# ===== AVALIAÇÃO NO TESTE: REWARD FULL + RETORNO PURO =====
portfolio_log_ret = []        # reward completo (logret - dd - custo)
portfolio_log_ret_puro = []   # logret puro (sem penalização)

with torch.no_grad():
    done = False
    while not done:
        # 1) política escolhe ação
        logits = policy(state)
        dist = Categorical(logits=logits)
        action = dist.sample()

        # 2) aplica ação e avança o ambiente de teste
        next_state, reward, done = env_test.step(
            aplicar_acao_portfolio(env_test.pesos, action.item())
        )

        # 3) computa log-retorno PURO (sem dd, sem custo)
        r_t = ret_test.iloc[env_test.t - 1].values  # retornos do dia atual
        r_p = float(np.dot(env_test.pesos, r_t))

        if 1 + r_p <= 0:
            log_ret_puro = -10.0
        else:
            log_ret_puro = np.log(1 + r_p)

        # 4) salva as métricas
        portfolio_log_ret.append(reward)            # reward FULL
        portfolio_log_ret_puro.append(log_ret_puro) # só log-ret

        # 5) atualiza o estado
        state = torch.FloatTensor(next_state).unsqueeze(0)

total_reward_test = np.sum(portfolio_log_ret)
avg_reward_test = np.mean(portfolio_log_ret)

print("\n===== AVALIAÇÃO NO TESTE (REWARD FULL) =====")
print(f"Total reward (teste) = {total_reward_test:.6f}")
print(f"Média diária (teste) = {avg_reward_test:.6f}")
print(f"N dias de teste = {len(portfolio_log_ret)}")

print("\n===== AVALIAÇÃO NO TESTE (RETORNO PURO) =====")
print(f"Total log-ret puro = {np.sum(portfolio_log_ret_puro):.6f}")
print(f"Média diária log-ret puro = {np.mean(portfolio_log_ret_puro):.6f}")
print(f"N dias de teste = {len(portfolio_log_ret_puro)}")





Arquivo encontrado! Carregando dados locais...
            BOVA11.SA  SMAL11.SA  IVVB11.SA  PETR4.SA   VALE3.SA  ITUB4.SA  \
Date                                                                         
2015-01-01        NaN        NaN        NaN       NaN        NaN       NaN   
2015-01-02  47.259998  52.020000  55.799999  2.572509  10.505502  9.372999   
2015-01-03        NaN        NaN        NaN       NaN        NaN       NaN   
2015-01-04        NaN        NaN        NaN       NaN        NaN       NaN   
2015-01-05  46.320000  50.549999  55.750000  2.352637  10.347521  9.420101   

                   SPY        QQQ         IWM        EEM         GLD  \
Date                                                                   
2015-01-01         NaN        NaN         NaN        NaN         NaN   
2015-01-02  171.093704  94.906532  103.315140  30.789457  114.080002   
2015-01-03         NaN        NaN         NaN        NaN         NaN   
2015-01-04         NaN        NaN         NaN 

c:\Users\olazu\OneDrive\Documentos\Python\portfolioDeAtivos\portfolioDeAtivos\teste.py:154: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  retornos = precos.pct_change().fillna(0)
c:\Users\olazu\OneDrive\Documentos\Python\portfolioDeAtivos\portfolioDeAtivos\teste.py:154: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  retornos = precos.pct_change().fillna(0)


[PPO] Iter 0 | rollout_reward=-0.020023 | avg_reward=-0.000078 | avg_return=0.499928 | avg_adv=0.0000
[PPO] Iter 1 | rollout_reward=0.044820 | avg_reward=0.000175 | avg_return=0.440698 | avg_adv=0.0000
[PPO] Iter 2 | rollout_reward=0.260921 | avg_reward=0.001019 | avg_return=0.374021 | avg_adv=-0.0000
[PPO] Iter 3 | rollout_reward=0.152864 | avg_reward=0.000597 | avg_return=0.316918 | avg_adv=0.0000
[PPO] Iter 4 | rollout_reward=0.124182 | avg_reward=0.000485 | avg_return=0.302382 | avg_adv=-0.0000
[PPO] Iter 5 | rollout_reward=0.047876 | avg_reward=0.000187 | avg_return=0.239331 | avg_adv=-0.0000
[PPO] Iter 6 | rollout_reward=0.101755 | avg_reward=0.000397 | avg_return=0.214473 | avg_adv=-0.0000
[PPO] Iter 7 | rollout_reward=0.196017 | avg_reward=0.000766 | avg_return=0.199145 | avg_adv=0.0000
[PPO] Iter 8 | rollout_reward=1.168630 | avg_reward=0.004565 | avg_return=0.229831 | avg_adv=-0.0000
[PPO] Iter 9 | rollout_reward=0.402589 | avg_reward=0.001573 | avg_return=0.204001 | avg_adv=